In [1]:
# # # # !pip install wandb
# # # !pip install transformers
# # !pip install datasets
# !pip install evaluate

In [1]:
# Import packages
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
# import wandb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import ElectraForSequenceClassification, ElectraTokenizer, Trainer, TrainingArguments
import torch.nn as nn
from transformers import ElectraForSequenceClassification, ElectraTokenizer, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import load_dataset, DatasetDict
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict, load_metric
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

/common/home/projectgrps/IS424/IS424G10/jupyterlab-venv-pytorch/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-01 13:52:12.214248: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-01 13:52:13.068128: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/apps/software/CUDA/11.7.0/nvvm/lib64:/opt/apps/software/CUDA/11.7.0/extras/CUPTI/lib64:/opt/apps/so

In [2]:
# Specify GPU
device = torch.device("cuda")

In [3]:
device

device(type='cuda')

Set some constants

In [4]:
import random
def set_random_seed(seed):
    SEED = seed
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [5]:
EPOCHS = 3
BATCH_SIZE = 32
LEARNING_RATE = 1e-5
set_random_seed(0)
MODEL_SAVE_PATH = "electra_ys"
MODEL_CHECKPOINT_PATH = "electra_checkpoint"

# Load and split dataset

In [6]:
# Load dataset
df = pd.read_csv('preprocessed.csv')
df.head()

,text,class,char_length,token_length
0,sex wife threatening suicide recently left wif...,1,364,54
1,weird affected compliment coming know girl fee...,0,76,11
2,finally thousand hear thousand bad year swear ...,0,66,10
3,need help help hard,1,33,6
4,ism lost hello adam sixteen 've struggling yea...,1,1376,193


In [7]:
del df['char_length']
del df['token_length']

In [8]:
df.head()

,text,class
0,sex wife threatening suicide recently left wif...,1
1,weird affected compliment coming know girl fee...,0
2,finally thousand hear thousand bad year swear ...,0
3,need help help hard,1
4,ism lost hello adam sixteen 've struggling yea...,1


In [9]:
# Split dataset into train, validation and test sets
train, test = train_test_split(df,
                               random_state=0,
                               test_size=0.2,
                               stratify=df['class'])

train, val = train_test_split(train,
                             random_state=0,
                             test_size=0.125,
                             stratify=train['class'])

In [10]:
train_data = train["text"].values.tolist()
train_labels = train["class"].values.tolist()

In [11]:
val_data = val["text"].values.tolist()
val_labels = val["class"].values.tolist()

In [12]:
test_data = test["text"].values.tolist()
test_labels = test["class"].values.tolist()

# Load the pre-trained model

https://huggingface.co/docs/transformers/model_doc/electra

https://huggingface.co/course/chapter3/3?fw=pt#training

In [13]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/electra-base-discriminator")

In [14]:
train_encodings = tokenizer(train_data, truncation=True, padding=True)
val_encodings = tokenizer(val_data, truncation=True, padding=True)
test_encodings = tokenizer(test_data, truncation=True, padding=True)

In [15]:
# import torch
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [16]:
train_dataset = Dataset(train_encodings, train_labels)
val_dataset = Dataset(val_encodings, val_labels)
test_dataset = Dataset(test_encodings, test_labels)

In [17]:
import numpy as np
import evaluate

metric_accuracy  = evaluate.load("accuracy")
metric_precision = evaluate.load("precision")
metric_recall = evaluate.load("recall")
metric_f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = metric_accuracy.compute(predictions=predictions, references=labels)
    precision = metric_precision.compute(predictions=predictions, references=labels)
    recall = metric_recall.compute(predictions=predictions, references=labels)
    f1 = metric_f1.compute(predictions=predictions, references=labels)
    
    return {"Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1": f1}

In [18]:
# Set training parameters
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    evaluation_strategy="steps",
    learning_rate=LEARNING_RATE,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    seed=0,
)

# Load the pretrained model.
# The ".to('cuda')" method moves the model into GPU. 
# If your machine does't have a GPU, please comment out ".to('cuda')" from this line.
model = AutoModelForSequenceClassification.from_pretrained("google/electra-base-discriminator", num_labels=2).to('cuda')

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.o

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
# Train the text classification model.
trainer.train()

/common/home/projectgrps/IS424/IS424G10/jupyterlab-venv-pytorch/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 162392
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 15225
  Number of trainable parameters = 109483778
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: ystan98. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.309600,0.223053,{'accuracy': 0.9183585499374973},{'precision': 0.9254097642212289},{'recall': 0.910093957417464},{'f1': 0.9176879617557584}
1000,0.211400,0.218066,{'accuracy': 0.9255140307771886},{'precision': 0.9382935274793572},{'recall': 0.9109559520730971},{'f1': 0.9244226731980405}
1500,0.208100,0.204233,{'accuracy': 0.9304711409974568},{'precision': 0.924804355222865},{'recall': 0.9371605896043445},{'f1': 0.9309414736481569}
2000,0.186100,0.186534,{'accuracy': 0.9325401956980904},{'precision': 0.9407166695942385},{'recall': 0.923282475648651},{'f1': 0.9319180406316613}
2500,0.185600,0.177520,{'accuracy': 0.9359886201991465},{'precision': 0.9220627503337784},{'recall': 0.9525040944746143},{'f1': 0.9370362518549926}
3000,0.178200,0.180793,{'accuracy': 0.9364627785680417},{'precision': 0.9264},{'recall': 0.9482803206620118},{'f1': 0.9372124723121487}
3500,0.168100,0.184944,{'accuracy': 0.9394370447002026},{'precision': 0.9306470687616152},{'recall': 0.949659512111025},{'f1': 0.9400571696744742}
4000,0.168200,0.178206,{'accuracy': 0.9396094659252554},{'precision': 0.9382949467170849},{'recall': 0.9411257650202569},{'f1': 0.9397082239531781}
4500,0.160000,0.165307,{'accuracy': 0.9406870985818354},{'precision': 0.9404669595933488},{'recall': 0.9409533660891303},{'f1': 0.9407100999655291}
5000,0.157900,0.167923,{'accuracy': 0.9409888357256778},{'precision': 0.9442514762070163},{'recall': 0.9373329885354711},{'f1': 0.9407795129125752}


***** Running Evaluation *****
  Num examples = 23199
  Batch size = 32
Trainer is attempting to log a value of "{'accuracy': 0.9183585499374973}" of type <class 'dict'> for key "eval/Accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9254097642212289}" of type <class 'dict'> for key "eval/Precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.910093957417464}" of type <class 'dict'> for key "eval/Recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9176879617557584}" of type <class 'dict'> for key "eval/F1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkp

TrainOutput(global_step=15225, training_loss=0.14811112502525592, metrics={'train_runtime': 16151.0181, 'train_samples_per_second': 30.164, 'train_steps_per_second': 0.943, 'total_flos': 1.2818139150606336e+17, 'train_loss': 0.14811112502525592, 'epoch': 3.0})

In [25]:
trainer.save_model('electra_ys')

Saving model checkpoint to electra_ys
Configuration saved in electra_ys/config.json
Model weights saved in electra_ys/pytorch_model.bin


In [20]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 23199
  Batch size = 32


Trainer is attempting to log a value of "{'accuracy': 0.9452131557394715}" of type <class 'dict'> for key "eval/Accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9444922547332186}" of type <class 'dict'> for key "eval/Precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.9460391345573658}" of type <class 'dict'> for key "eval/Recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9452650617975109}" of type <class 'dict'> for key "eval/F1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.17375293374061584,
 'eval_Accuracy': {'accuracy': 0.9452131557394715},
 'eval_Precision': {'precision': 0.9444922547332186},
 'eval_Recall': {'recall': 0.9460391345573658},
 'eval_F1': {'f1': 0.9452650617975109},
 'eval_runtime': 179.1751,
 'eval_samples_per_second': 129.477,
 'eval_steps_per_second': 4.046,
 'epoch': 3.0}

In [28]:
trainer.predict(test_dataset).metrics

***** Running Prediction *****
  Num examples = 46398
  Batch size = 32


{'test_loss': 0.15672726929187775,
 'test_Accuracy': {'accuracy': 0.948855554118712},
 'test_Precision': {'precision': 0.9505147504109352},
 'test_Recall': {'recall': 0.9470327112873336},
 'test_F1': {'f1': 0.9487705360419681},
 'test_runtime': 351.0832,
 'test_samples_per_second': 132.157,
 'test_steps_per_second': 4.13}

In [27]:
saved_model = AutoModelForSequenceClassification.from_pretrained('electra_ys')

loading configuration file electra_ys/config.json
Model config ElectraConfig {
  "_name_or_path": "electra_ys",
  "architectures": [
    "ElectraForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file electra_ys/pytorch_model.bin
All model checkpoint weights were used

In [100]:
# Label 0 = positive, label 1 = suicidal

In [103]:
test_text = "i wish i could die right now"

In [104]:
unseen_encodings = tokenizer(test_text, truncation=True, padding=True,return_tensors="pt")

In [105]:
saved_model.to('cuda')
# Pass your inputs to the model and return the logits:
with torch.no_grad():
    logits = saved_model(**unseen_encodings.to('cuda')).logits

In [106]:
probabilities = torch.sigmoid(logits)
predicted_class_id = probabilities.argmax().item()
model.config.id2label[predicted_class_id]

'LABEL_1'

In [108]:
test_text = "i wish i could die right now. just kidding haha"

In [109]:
unseen_encodings = tokenizer(test_text, truncation=True, padding=True,return_tensors="pt")
saved_model.to('cuda')
# Pass your inputs to the model and return the logits:
with torch.no_grad():
    logits = saved_model(**unseen_encodings.to('cuda')).logits
probabilities = torch.sigmoid(logits)
predicted_class_id = probabilities.argmax().item()
model.config.id2label[predicted_class_id]

'LABEL_0'